In [9]:
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
import pinecone
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENVIRONMENT")

# Pinecone 클라이언트 초기화
pc = PineconeClient(api_key=pinecone_api_key, environment=pinecone_env)

# 사용할 인덱스 이름과 임베딩 차원 설정, 10_create_DB_pinecone.py에서 정의한 것과 동일
index_name = "aws-embeddings-demo"
embedding_dim = 1536  # text-embedding-3-small의 벡터 차원

# 인덱스 생성 (없으면 새로 생성, 이미 존재하면 넘어감)
if not pc.has_index(index_name):
    pc.create_index(name=index_name, dimension=embedding_dim)

# 인덱스 객체 연결
index = pc.Index(index_name)
# 인덱스 상태 확인
print(f"Index '{index_name}' is ready: {index.describe_index_stats()}")

Index 'aws-embeddings-demo' is ready: {'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3,
 'vector_type': 'dense'}


In [10]:
# OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [11]:
from langchain_core.documents import Document

# 예시 문서 생성 (내용과 메타데이터)
doc1 = Document(page_content="Building an exciting new project with LangChain - come check it out!",
                metadata={"source": "tweet"})
doc2 = Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
                metadata={"source": "news"})
doc3 = Document(page_content="LangGraph is the best framework for building stateful, agentic applications!",
                metadata={"source": "tweet"})

# 벡터스토어 초기화 (PineconeVectorStore에 Pinecone 인덱스와 임베딩 객체 연결)
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# 문서들을 벡터 임베딩하여 Pinecone에 저장
vector_store.add_documents([doc1, doc2, doc3])
print(f"현재 벡터 DB 내 벡터 개수: {index.describe_index_stats()['total_vector_count']}")  # 벡터 총량 출력

현재 벡터 DB 내 벡터 개수: 6
